In [1]:
# T# ====================================================
# VETERINARY ANIMAL CLASSIFICATION WITH TRANSFORMERS
# Dataset: Animals-10 (Kaggle)
# Model: Vision Transformer (ViT) - No Pretrained Weights
# Accuracy Target: >85%
# Domain: Veterinary (Animal Health & Species Identification)
# SDG Alignment: Life on Land (SDG 15)
# ====================================================

# Install required packages
!pip install torch torchvision matplotlib seaborn scikit-learn tqdm pillow pandas -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

   
            
       

    
   

Using device: cuda
GPU available: True
GPU: Tesla P100-PCIE-16GB


In [2]:
# ====================================================
# VETERINARY ANIMAL CLASSIFICATION - FIXED VERSION
# Dataset: Animals-10 (Kaggle)
# Model: Custom Vision Transformer (No Pretrained Weights)
# ====================================================

# Install packages
!pip install torch torchvision matplotlib seaborn scikit-learn tqdm pillow pandas -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ====================================================
# 1. LOAD DATASET (FIXED - NO SHELL COMMANDS IN PYTHON INDENTATION)
# ====================================================

print("="*50)
print("LOADING ANIMALS-10 DATASET")
print("="*50)

# Dataset paths to try
dataset_paths = [
    "/kaggle/input/animals10/animals10",
    "/kaggle/input/animals10",
    "/kaggle/input/animals-10",
    "/kaggle/input/animal10"
]

data_dir = None
for path in dataset_paths:
    if os.path.exists(path):
        data_dir = path
        print(f"✓ Found dataset at: {path}")
        break

# If not found, show instructions
if data_dir is None:
    print("\n❌ Dataset not found.")
    print("\n📋 PLEASE ADD THIS DATASET:")
    print("1. Click '+ Add Data' button on Kaggle")
    print("2. Search for 'animals10'")
    print("3. Use: https://www.kaggle.com/datasets/alessiocorrado99/animals10")
    print("\nUsing synthetic data for demo...")
    # Create demo data
    from torchvision.datasets import FakeData
    full_dataset = FakeData(size=2000, image_size=(3, 224, 224), num_classes=10, 
                           transform=transforms.ToTensor())
else:
    # Load real dataset
    print(f"\nLoading from: {data_dir}")
    
    # Check what's in the directory (using Python, not shell commands)
    print("\nExploring directory structure...")
    try:
        items = os.listdir(data_dir)
        print(f"Found {len(items)} items in directory")
        
        # Show first 10 items
        for i, item in enumerate(items[:10]):
            item_path = os.path.join(data_dir, item)
            if os.path.isdir(item_path):
                sub_items = os.listdir(item_path)[:3]
                print(f"  {item}/ [directory] - sample: {sub_items}")
            else:
                print(f"  {item} [file]")
        
        # Load with ImageFolder
        print("\nLoading images...")
        train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
        print(f"✓ Loaded {len(full_dataset)} images")
        print(f"✓ Classes: {len(full_dataset.classes)}")
        print(f"✓ Class names: {full_dataset.classes}")
        
        # Show class distribution
        print("\nClass distribution:")
        class_counts = {}
        for _, label in full_dataset.samples:
            class_name = full_dataset.classes[label]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        for cls, count in class_counts.items():
            print(f"  {cls}: {count} images")
            
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Using synthetic data instead...")
        from torchvision.datasets import FakeData
        full_dataset = FakeData(size=2000, image_size=(3, 224, 224), num_classes=10, 
                               transform=transforms.ToTensor())


 


Device: cuda
GPU: Tesla P100-PCIE-16GB
LOADING ANIMALS-10 DATASET
✓ Found dataset at: /kaggle/input/animals10

Loading from: /kaggle/input/animals10

Exploring directory structure...
Found 2 items in directory
  translate.py [file]
  raw-img/ [directory] - sample: ['cavallo', 'pecora', 'elefante']

Loading images...
✓ Loaded 26179 images
✓ Classes: 1
✓ Class names: ['raw-img']

Class distribution:
  raw-img: 26179 images


In [3]:
# ====================================================
# 2. DATA TRANSFORMS AND SPLITTING
# ====================================================

print("\n" + "="*50)
print("PREPARING DATA")
print("="*50)

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])
# Split dataset
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

print(f"Dataset split:")
print(f"  Train: {len(train_dataset)} images")
print(f"  Val: {len(val_dataset)} images")
print(f"  Test: {len(test_dataset)} images")
# Apply transforms to val/test
class TransformedDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        img, label = self.subset[index]
        if self.transform:
            img = self.transform(img)
        return img, label
        
    def __len__(self):
        return len(self.subset)
        val_dataset = TransformedDataset(val_dataset, val_transform)
test_dataset = TransformedDataset(test_dataset, val_transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"\n✓ Data loaders created (batch size: {batch_size})")



PREPARING DATA
Dataset split:
  Train: 18325 images
  Val: 3926 images
  Test: 3928 images

✓ Data loaders created (batch size: 32)


In [4]:
# ====================================================
# 3. CUSTOM CNN MODEL - FIXED VERSION
# ====================================================

print("\n" + "="*50)
print("BUILDING MODEL")
print("="*50)

import torch
import torch.nn as nn

class AnimalClassifierCNN(nn.Module):
    def __init__(self, num_classes=10, input_size=224):
        super(AnimalClassifierCNN, self).__init__()
        
        # -----------------
        # Feature extractor
        # -----------------
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
        )

        # -----------------
        # Dynamic feature size calculation
        # -----------------
        with torch.no_grad():
            dummy = torch.zeros(1, 3, input_size, input_size)
            dummy_out = self.features(dummy)
            self.flatten_dim = dummy_out.view(1, -1).shape[1]

        # -----------------
        # Classifier
        # -----------------
        self.classifier = nn.Sequential(
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)   # Flatten
        x = self.classifier(x)
        return x


# -----------------
# Create model
# -----------------
num_classes = 10
if hasattr(full_dataset, 'classes'):
    num_classes = len(full_dataset.classes)

model = AnimalClassifierCNN(num_classes=num_classes, input_size=224)  # change 224 if needed
model = model.to(device)

print(f"Model created with {num_classes} classes")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")



BUILDING MODEL
Model created with 1 classes
Total parameters: 51,800,225


In [5]:
# ====================================================
# 4. TRAINING SETUP - KAGGLE READY
# ====================================================

import torch
import torch.nn as nn
import torch.optim as optim

print("\n" + "="*50)
print("SETTING UP TRAINING")
print("="*50)

# -----------------
# Device
# -----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -----------------
# Loss, Optimizer, Scheduler
# -----------------
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Kaggle-safe scheduler (no verbose arg)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    patience=2,
    factor=0.5
)

# -----------------
# Training Function
# -----------------
def train_model(model, train_loader, val_loader, epochs=15):
    
    best_acc = 0.0
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("=" * 40)
        
        # =====================
        # Training
        # =====================
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        train_acc = 100.0 * correct / total
        train_loss = train_loss / len(train_loader)
        
        # =====================
        # Validation
        # =====================
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        val_acc = 100.0 * correct / total
        val_loss = val_loss / len(val_loader)
        
        # =====================
        # Scheduler step
        # =====================
        scheduler.step(val_acc)
        
        # =====================
        # History
        # =====================
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # =====================
        # Logs
        # =====================
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
        print(f"LR: {optimizer.param_groups[0]['lr']}")
        
        # =====================
        # Save Best Model
        # =====================
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'history': history
            }, 'best_model.pth')
            
            print(f"✅ Best model saved! Val Acc: {val_acc:.2f}%")
        
        # =====================
        # Early Stopping
        # =====================
        if val_acc >= 90:
            print("🎯 Target accuracy reached. Early stopping.")
            break
    
    print("\nTraining Complete ✅")
    return history
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=20
)



SETTING UP TRAINING
Device: cuda

Epoch 1/20
Train Loss: 0.0000 | Train Acc: 100.00%
Val   Loss: 0.0000 | Val   Acc: 100.00%
LR: 0.001
✅ Best model saved! Val Acc: 100.00%
🎯 Target accuracy reached. Early stopping.

Training Complete ✅


In [6]:
# ====================================================
# 5. TRAIN MODEL
# ====================================================

print("\n" + "="*50)
print("TRAINING MODEL")
print("="*50)

history = train_model(model, train_loader, val_loader, epochs=15)


TRAINING MODEL

Epoch 1/15
Train Loss: 0.0000 | Train Acc: 100.00%
Val   Loss: 0.0000 | Val   Acc: 100.00%
LR: 0.001
✅ Best model saved! Val Acc: 100.00%
🎯 Target accuracy reached. Early stopping.

Training Complete ✅


In [7]:
# ====================================================
# STEP 6: ULTRA-SIMPLE MODEL (NO INDENTATION ERRORS)
# ====================================================

print("\n" + "="*50)
print("BUILDING SIMPLE CNN MODEL")
print("="*50)

class SimpleAnimalClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleAnimalClassifier, self).__init__()
        
        # All layers defined separately
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.drop1 = nn.Dropout2d(0.25)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.drop2 = nn.Dropout2d(0.25)
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.drop3 = nn.Dropout2d(0.25)
        
        # Flattened size calculation
        self.flattened_size = 128 * 28 * 28  # 224/2/2/2 = 28
        
        # Fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(512, 256)
        self.relu5 = nn.ReLU()
        self.drop5 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.drop1(x)
        
        # Layer 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.drop2(x)
        
        # Layer 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.drop3(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.drop4(x)
        
        x = self.fc2(x)
        x = self.relu5(x)
        x = self.drop5(x)
        
        x = self.fc3(x)
        
        return x

# Create model
num_classes = len(class_names) if 'class_names' in locals() else 10
model = SimpleAnimalClassifier(num_classes=num_classes)
model = model.to(device)

print(f"✅ Simple model created with {num_classes} classes")
print(f"✅ Total parameters: {sum(p.numel() for p in model.parameters()):,}")


BUILDING SIMPLE CNN MODEL
✅ Simple model created with 10 classes
✅ Total parameters: 51,608,330


In [8]:
from torchvision import transforms

# -----------------------------
# STEP 0: SAFE TRANSFORM
# -----------------------------
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),      # resize images
    # transforms.ToTensor(),            # ❌ REMOVE this line!
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# -----------------------------
# STEP 1: ASSIGN TRANSFORM TO DATASET
# -----------------------------
test_dataset.transform = test_transform   # overwrite old transform

# -----------------------------
# STEP 2: CREATE TEST LOADER
# -----------------------------
from torch.utils.data import DataLoader

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("✅ Test loader ready with Tensor-safe transform")


✅ Test loader ready with Tensor-safe transform
